In [3]:
import pandas as pd
file_path = pd.read_csv("train.csv")
audio_files = file_path["path"]

In [9]:
audio_files[0]

'data/rawaudio/pgLXVFvo5GI_27.wav'

In [14]:
import librosa 
_ , sr  = librosa.load(f"../{audio_files[0]}")

In [6]:
audio_files = audio_files[:66676]
len(audio_files)

66676

In [7]:
import torch
import os
import laion_clap
from tqdm import tqdm

/home/gaurav/Desktop/projects/IE643_Novelty/novelty/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-25 15:38:33.688993: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = laion_clap.CLAP_Module(enable_fusion=False).to(device)
model.load_ckpt()

/home/gaurav/Desktop/projects/IE643_Novelty/novelty/lib/python3.12/site-packages/torch/functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4317.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
Download completed!
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm2.weight 	 Loaded
aud

In [ ]:
batch_size = 64 
embeddings = []

for i in tqdm(range(0, len(audio_files), batch_size)):
    batch_paths = audio_files[i:i+batch_size]
    batch_paths = ["../" + f for f in batch_paths]
    audio_emb = model.get_audio_embedding_from_filelist(
        batch_paths,
        use_tensor=True
    ).to(device)

    embeddings.append(audio_emb.cpu())  # store on CPU to free GPU memory

# combine all embeddings
embeddings = torch.cat(embeddings, dim=0)
print("Final embedding shape:", embeddings.shape)

  0%|          | 0/1042 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 288.00 MiB. GPU 0 has a total capacity of 3.68 GiB of which 276.31 MiB is free. Including non-PyTorch memory, this process has 3.39 GiB memory in use. Of the allocated memory 2.98 GiB is allocated by PyTorch, and 316.23 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 

In [ ]:
import torch
import laion_clap

device = "cuda" if torch.cuda.is_available() else "cpu"

model = laion_clap.CLAP_Module(enable_fusion=False).to(device)
model.load_ckpt()   # downloads pretrained weights

audio_paths = ["/content/car-engine-335601.mp3"]

audio_emb = model.get_audio_embedding_from_filelist(
    ["/content/car-engine-335601.mp3"],
    use_tensor=True
).to(device)


print(audio_emb.shape)   # e.g. torch.Size([2, 512])

In [ ]:
audio_paths = "/home/gaurav/Desktop/projects/IE643_Novelty/data/rawaudio"
